# Exercise: Bild a Mixed View

## Set correct directory

In [1]:
cd ..

/bptk-py


## Import libraries

In [3]:
import pandas as pd
import time
import pickle
import json
from elasticsearch import Elasticsearch
import os.path
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntSlider

## Example Query of the batch index

In [4]:
es = Elasticsearch([{'host': 'es_node1', 'port': 9200}])
es.search(index="1_car_batch",body={"size": 0,
                "aggs" : {
                    "revenue" : {
                        "date_range": {
                            "field": "time",
                            "ranges": [
                                { "from": 1440,
                                  "to": 1441 } 
                            ]
                        },
                        "aggs": {
                            "total_revenue": {
                              "sum": {
                                "field": "sum_revenue"
                              }
                            }
                        }
                    }
                }})

{'took': 392,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 11, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'revenue': {'buckets': [{'key': '1440.0-1441.0',
     'from': 1440.0,
     'to': 1441.0,
     'doc_count': 1,
     'total_revenue': {'value': 253.60000610351562}}]}}}

## Mixed Query Function

In [8]:
def es_mixed_query(car_index,query_time):
    
    # add your code here
    print("please add your <CODE>")
    
    batch_result_value = 0
    rt_result_value =0
    full_scan_result_value = 0
    car_result_value = 0
    mixed_result_value = batch_result_value+ rt_result_value
    difference_mixed_full_scan = full_scan_result_value - mixed_result_value
    difference_mixed_car = car_result_value - mixed_result_value
    print("Mixed result: "+str(mixed_result_value))
    print("Car result: "+str(car_result_value))
    print("Full scan result: "+str(full_scan_result_value))
    print("Difference Mixed/Full Scan: "+str( difference_mixed_full_scan))
    print("Difference Mixed/Car: "+str( difference_mixed_car))

In [6]:
es_mixed_query("1_car",14400)

Mixed result: 0
Car result: 0
Full scan result: 0
Difference Mixed/Full Scan: 0
Difference Mixed/Car: 0


## Mixed Query Using Batch and Real Time Views

In [7]:
# read last batch processing time
time_file_object = open("csv/sim_time.pickle",'rb')  
last_batch_processing_time = pickle.load(time_file_object)

# obtain number of cars
with open('scenarios/abm.json') as scenario_file:
    abm = json.load(scenario_file)

num_cars=abm["CARMODEL"]["scenarios"]["scenario"]["agents"][1]["count"]
car_tuples = [(str(i)+"_car",str(i)+"_car") for i in range(1,num_cars+1)]

car_index=widgets.Dropdown(
    options=car_tuples,
    value='1_car',
    description='choose car:'
)

query_time=widgets.IntSlider(min=0, max=last_batch_processing_time+1440, step=1, description='query time:')

run_button = widgets.Button(description="Run Query")
refresh_button = widgets.Button(description="Refresh")

output = widgets.Output()

display(car_index,query_time,run_button,refresh_button,output)

def on_run_clicked(b):
    with output:
        output.clear_output()
        es_mixed_query(car_index.value,query_time.value)
    
def on_refresh_clicked(b):
    with output:
        output.clear_output()
        time_file_object = open("csv/sim_time.pickle",'rb')  
        last_batch_processing_time = pickle.load(time_file_object)
        query_time.max=last_batch_processing_time+1440
        

run_button.on_click(on_run_clicked)
refresh_button.on_click(on_refresh_clicked)

Dropdown(description='choose car:', options=(('1_car', '1_car'), ('2_car', '2_car'), ('3_car', '3_car')), valu…

IntSlider(value=0, description='query time:', max=17280)

Button(description='Run Query', style=ButtonStyle())

Button(description='Refresh', style=ButtonStyle())

Output()